Essential Packages

In [1]:
%pip install  groq


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install groq pdfplumber pytesseract watchdog python-dotenv pdf2image python-docx docx2txt 


In [3]:
# %pip install "pip<24.1"
# %pip install textract


In [4]:
%pip install --upgrade pymupdf

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


API connections

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

In [18]:
import os 
import groq

groq.api_key = GROQ_API_KEY

Extracting files

In [19]:
import pdfplumber

In [20]:
import groq

In [59]:
from groq import Groq
client = Groq(api_key=groq.api_key)


summary_dict = {}

folder_queue = "resumes/queue"
for filename in os.listdir(folder_queue):
    filepath = os.path.join(folder_queue,filename)
    if os.path.isfile(filepath) and filename.lower().endswith((".pdf")):
        print("Processing file:", filename)
        doc = pdfplumber.open(f"/home/sneeraj/summariser/resumes/queue/{filename}")
        for page in doc.pages:
            text = page.extract_text() + "\n"
            # print(text)
            prompt = text + "\n tl;dr:"
            # print(len(text), text[:500])
            response = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages = [
                    {"role": "system",
                    "content" : (
                        "You are a AI assistant that extracts resume information from a resume  "
                        "always return json dont inclue any other text"
                        
                    )},
                    {"role": "user",
                    "content" : (
                        "first check if the pdf is valid or not and mark the status as valid or invalid if invalid u dont have to extract details just return the status in that json file"
                        f"Here is the resume \n\n {text}\n\n return this in json formate : \n and summarise at the end "
                            "{\n"
                            '  "status": "valid" or "invalid",\n'
                            '  "name": "",\n'
                            '  "email": "",\n'
                            '  "phone": "",\n'
                            '  "location": "",\n'
                            '  "education": [],\n'
                            '  "experience": [],\n'
                            '  "skills": []\n'
                            '  "summary": "",'
                            "}"

                    )
                    }
                ],
                temperature=0.0,
                max_tokens=500,
            )
    summary_dict[filename] = response.choices[0].message.content

Processing file: 002.pdf


In [11]:
import json
status = json.loads(summary_dict["001.pdf"])["status"]
print(status)

valid


In [29]:
%pip install pinecone

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# conda install -c conda-forge ipywidgets


2 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/sneeraj/miniconda3/envs/summariser

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipywidgets-8.1.7           |     pyhd8ed1ab_0         112 KB  conda-forge
    jupyterlab_widgets-3.0.15  |     pyhd8ed1ab_0         185 KB  conda-forge
    widgetsnbextension-4.0.14  |     pyhd8ed1ab_0         868 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following NEW packages wil

In [54]:
import os
from pinecone import Pinecone, ServerlessSpec
from groq import Groq
import json

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key = PINECONE_API_KEY)


index_name = "resumes"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension = 384,
        metric = "cosine",
        spec=ServerlessSpec(
        cloud="aws",      # or "gcp"
        region="us-east-1"
        )
    )
index = pc.Index(index_name)





In [52]:
%pip install sentence-transformers


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
import shutil
import json
import traceback

from sentence_transformers import SentenceTransformer
hf_model = SentenceTransformer("all-MiniLM-L6-v2")  


folder_processed ="resumes/processed"
folder_rejected ="resumes/rejected"

for filename in summary_dict.keys():
    resume_str = summary_dict[filename]
    cleaned_str = resume_str.strip().strip("```json").strip("```")
    try:
        if not cleaned_str:
            print(f"Skipping file: {filename}")
            continue

        resume = json.loads(cleaned_str)
        status = resume.get("status", "invalid")

        if status == 'valid':
            text_to_embed = resume["summary"] + "Skills : " + str(resume["skills"])

            embedding = hf_model.encode(text_to_embed).tolist()
        
            metadata = {
                "name": resume["name"],
                "email": resume["email"],
                "phone": resume["phone"],
                "location": resume["location"],
                "skills": str(resume["skills"]),
                "summary": resume["summary"]
            }

            index.upsert([
                {
                    "id" :filename.split('.')[0],
                    "values" : embedding,
                    "metadata" : metadata
                }
                ])
            shutil.move(f"{folder_queue}/{filename.split('.')[0]}.pdf",f"{folder_processed}/{filename.split('.')[0]}.pdf")
        elif status == 'invalid' : 
            shutil.move(f"{folder_queue}/{filename.split('.')[0]}.pdf",f"{folder_rejected}/{filename.split('.')[0]}.pdf")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for file: {filename}")
        print(f"Invalid string content: {resume_str}")
        print(f"Error details: {e}")

    except Exception as e:
        print(f"An unexpected error occurred for file: {filename}")
        print(f"Invalid string content: {resume_str}")
        traceback.print_exc()



Error decoding JSON for file: 002.pdf
Invalid string content: ```
{
  "status": "valid",
  "name": "Clara Fernandez",
  "email": "clara.fernandez@email.com",
  "phone": "(123) 456-7890",
  "location": "San Antonio, TX",
  "education": [
    {
      "degree": "M.A.",
      "field": "English",
      "university": "University of Texas at San Antonio",
      "dates": "January 2021 - May 2023"
    },
    {
      "degree": "B.A.",
      "field": "English",
      "university": "Texas A&M University",
      "dates": "August 2016 - June 2020"
    }
  ],
  "experience": [
    {
      "title": "Public Relations Intern",
      "company": "The DeBerry Group",
      "dates": "July 2023 - Current",
      "achievements": [
        "Conducted market research to identify 24 media trends and competitor strategies",
        "Drafted 17 press releases, boosting client visibility in target markets by 36%",
        "Practiced keyword research to optimize 11 press releases for search engine visibility",
     

In [ ]:
# import json
# import shutil
# import traceback

# folder_processed = "resumes/processed"
# folder_rejected = "resumes/rejected"


# for filename in summary_dict.keys():
#     resume_str = summary_dict[filename]
    
#     # Strip the markdown code block notation using string methods
#     cleaned_str = resume_str.strip().strip("```json").strip("```")
    
#     try:
#         if not cleaned_str:
#             print(f"Skipping empty resume string for file: {filename}")
#             continue

#         resume = json.loads(cleaned_str)
        
#         # Safely access the status key using .get() to prevent a KeyError
#         status = resume.get("status", "unknown")
        
#         if status == 'valid':
#             # Example: Process valid resumes
#             text_to_embed = resume.get("summary", "") + "Skills : " + str(resume.get("skills", []))
#             print(f"Processing valid resume from {filename} with status: {status}")
            
#             # Now you can safely access other keys
#             name = resume.get("name", "N/A")
#             print(f"Name: {name}")

#         elif status == 'invalid':
#             # Example: Handle invalid resumes
#             print(f"Rejecting resume from {filename} with status: {status}")
#             # If you have the original file, you can move it
#             # shutil.move(f"{folder_queue}/{filename.split('.')}.pdf", f"{folder_rejected}/{filename.split('.')}.pdf")
            
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON for file: {filename}")
#         print(f"Invalid string content: {resume_str}")
#         print(f"Error details: {e}")
#     except Exception as e:
#         print(f"An unexpected error occurred for file: {filename}")
#         print(f"Invalid string content: {resume_str}")
#         traceback.print_exc()


Processing valid resume from 001.pdf with status: valid
Name: Neeraj Suriya
Processing valid resume from 002.pdf with status: valid
Name: Neeraj Suriya
